In [ ]:
import os
import sys
import json
import datetime

import pandas as pd
import numpy as np

In [ ]:
cwd = os.getcwd()
join = os.path.join
norm = os.path.normpath

In [ ]:
sys.path.append(norm(join(cwd, '..', '..', '..', 'glhe')))

In [ ]:
from standalone.plant_loop import PlantLoop
import glhe

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline

In [ ]:
plt.style.use('seaborn-bright')
plt.rcParams['figure.figsize'] = [7, 5]
plt.rcParams['font.size'] = 12

# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [ ]:
data_path = norm(join(cwd, '..', '..', 'validation', 'MFRTRT', 'MFRTRT_loads.csv'))
df_raw_data = pd.read_csv(data_path, parse_dates=True, index_col='Date/Time')

In [ ]:
df_sim_input_data = df_raw_data[['Inst. HT [1] [W]', 'mdot [kg/s]', 'Outlet 1 [C]', 'Inlet 1 [C]']].copy(deep=True)
df_sim_input_data.rename(columns = {'Inst. HT [1] [W]': 'Exp. HT Rate [W]', 
                                    'Outlet 1 [C]': 'Exp. Inlet [C]', 
                                    'Inlet 1 [C]': 'Exp. Outlet [C]'}, inplace=True)

In [ ]:
df_sim_input_data['time'] = pd.to_timedelta(df_sim_input_data.index)
df_sim_input_data['time'] = datetime.datetime(year=2019, month=1, day=1, hour=0, minute=0) + (df_sim_input_data['time'] - df_sim_input_data['time'][0])
df_sim_input_data.set_index('time', inplace=True)
df_sim_input_data.index.rename('Date/Time', inplace=True)

In [ ]:
timestep = 60

df_sim_input_data = df_sim_input_data.resample('{}S'.format(timestep)).mean()
df_sim_input_data.dropna(inplace=True)

df_sim_input_data.to_csv(join(cwd, 'input_data_high.csv'))

In [ ]:
d = glhe.utilities.functions.load_json('MFRTRT_STS_input.json')
d['ground-heat-exchanger'][0]['g-function-path'] = norm(join(cwd, 'g.csv'))
d['temperature-profile'][0]['path'] = norm(join(cwd, 'input_data_high.csv'))
d['flow-profile'][0]['path'] = norm(join(cwd, 'input_data_high.csv'))
d['simulation']['output-path'] = cwd
d['simulation']['output-csv-name'] = 'out_high.csv'

d['simulation']['time-step'] = timestep
d['simulation']['runtime'] = 3600 * 193
d['borehole-definitions'][0]['number-iterations'] = 2
d['borehole-definitions'][0]['fraction-grout'] = 0.333
d['borehole-definitions'][0]['segments'] = 1

In [ ]:
with open(join(cwd, 'in_high.json'), 'w') as f:
    f.write(json.dumps(d, sort_keys=True, indent=2, separators=(',', ': ')))

In [ ]:
PlantLoop('in_high.json').simulate()

In [ ]:
df = pd.read_csv('out_high.csv', index_col=0, parse_dates=True)

In [ ]:
df['Outlet Temp Error [C]'] = df['GroundHeatExchangerSTS:GHE 1:Outlet Temp. [C]'] - df_sim_input_data['Exp. Outlet [C]']
df['Heat Rate Error [W]'] = df['GroundHeatExchangerSTS:GHE 1:Heat Rate [W]'] - df_sim_input_data['Exp. HT Rate [W]']
df['DT Error [C]'] = (df['GroundHeatExchangerSTS:GHE 1:Inlet Temp. [C]'] - df['GroundHeatExchangerSTS:GHE 1:Outlet Temp. [C]']) - (df_sim_input_data['Exp. Inlet [C]'] - df_sim_input_data['Exp. Outlet [C]'])
df['Heat Rate Error [%]'] = (df['GroundHeatExchangerSTS:GHE 1:Heat Rate [W]'] - df_sim_input_data['Exp. HT Rate [W]']) / df_sim_input_data['Exp. HT Rate [W]'] * 100

In [ ]:
def plot_temps(chart_no):

    fig = plt.figure(dpi=200)
    ax = fig.add_subplot(211)
    
    if chart_no == 1:
        start_time = '2019-01-01 00:00:00'
        end_time = '2019-01-01 00:30:00'
    elif chart_no == 2:
        start_time = '2019-01-01 00:00:00'
        end_time = '2019-01-01 02:00:00'
    elif chart_no == 3:
        start_time = '2019-01-03 00:00:00'
        end_time = '2019-01-03 02:00:00'
    elif chart_no == 4:
        start_time = '2019-01-06 01:00:00'
        end_time = '2019-01-06 03:00:00'
    elif chart_no == 5:
        start_time = '2019-01-01 01:00:00'
        end_time = '2019-01-09 00:00:00'
        
    ax.plot(df_sim_input_data['Exp. Inlet [C]'].loc[start_time:end_time], label='Exp. Inlet')
    ax.plot(df['GroundHeatExchangerSTS:GHE 1:Inlet Temp. [C]'].loc[start_time:end_time], linestyle='--', label='Sim. Inlet')
    
    ax.plot(df_sim_input_data['Exp. Outlet [C]'].loc[start_time:end_time], label='Exp. Outlet')
    ax.plot(df['GroundHeatExchangerSTS:GHE 1:Outlet Temp. [C]'].loc[start_time:end_time], linestyle='--', label='Sim. Outlet')

    plt.ylabel('Temperature [C]')
    plt.grid()
    plt.legend()
    ax.legend(loc=0, ncol=2)
    
    if chart_no in [1, 2, 3, 4]:
        ax.xaxis.set_major_formatter(mdates.DateFormatter("%H-%M"))
    elif chart_no in [5]:
        ax.xaxis.set_major_formatter(mdates.DateFormatter("%d"))
        
    ax2 = fig.add_subplot(212)
    ax2.plot(df['DT Error [C]'].loc[start_time:end_time], label=r'$\Delta$T Error')

    if chart_no in [1, 2, 3, 4]:
        ax2.xaxis.set_major_formatter(mdates.DateFormatter("%H-%M"))
        plt.xlabel('Hr-Min')
    elif chart_no in [5]:
        ax2.xaxis.set_major_formatter(mdates.DateFormatter("%d"))
        plt.xlabel('Day')

    plt.ylabel('Error [C]')

    plt.grid()
    
    if chart_no == 1:
        plt.savefig('Temps_High_First-30min_ts-{}.PNG'.format(timestep), bbox_inches='tight')
    elif chart_no == 2:
        plt.savefig('Temps_High_First-2hr_ts-{}.PNG'.format(timestep), bbox_inches='tight')
    elif chart_no == 3:
        plt.savefig('Temps_High_First-FlowChange_ts-{}.PNG'.format(timestep), bbox_inches='tight')
    elif chart_no == 4:
        plt.savefig('Temps_High_Fourth-FlowChange_ts-{}.PNG'.format(timestep), bbox_inches='tight')
    elif chart_no == 5:
        plt.savefig('Temps_High_Full_ts-{}.PNG'.format(timestep), bbox_inches='tight')
    plt.show()

In [ ]:
plot_temps(1)

In [ ]:
plot_temps(2)

In [ ]:
plot_temps(3)

In [ ]:
plot_temps(4)

In [ ]:
plot_temps(5)

In [ ]:
data_path_low = norm(join(cwd, '..', '..', 'validation', 'MFRTRT', 'MFRTRT_Low_Flow_Loads.csv'))
df_raw_data_low = pd.read_csv(data_path_low, parse_dates=True, index_col='Date/Time')

In [ ]:
df_raw_data_low['time'] = pd.to_timedelta(df_raw_data_low.index)
df_raw_data_low['time'] = datetime.datetime(year=2019, month=1, day=1, hour=0, minute=0) + (df_raw_data_low['time'] - df_raw_data_low['time'][0])
df_raw_data_low.set_index('time', inplace=True)
df_raw_data_low.index.rename('Date/Time', inplace=True)

In [ ]:
df_raw_data_low = df_raw_data_low.resample('{}S'.format(timestep)).mean()
df_raw_data_low.dropna(inplace=True)

df_raw_data_low.to_csv(join(cwd, 'input_data_low.csv'))

In [ ]:
d_low = glhe.utilities.functions.load_json('MFRTRT_STS_input.json')
d_low['ground-heat-exchanger'][0]['g-function-path'] = norm(join(cwd, 'g.csv'))
d_low['temperature-profile'][0]['path'] = norm(join(cwd, 'input_data_low.csv'))
d_low['flow-profile'][0]['path'] = norm(join(cwd, 'input_data_low.csv'))
d_low['simulation']['output-path'] = cwd
d_low['simulation']['output-csv-name'] = 'out_low.csv'
d_low['simulation']['initial-temperature'] = 17.1
d_low['ground-temperature-model']['temperature'] = 17.1

d_low['simulation']['time-step'] = timestep
d_low['simulation']['runtime'] = 3600 * 68
d_low['borehole-definitions'][0]['number-iterations'] = 2
d_low['borehole-definitions'][0]['fraction-grout'] = 0.333
d_low['borehole-definitions'][0]['segments'] = 1

In [ ]:
with open(join(cwd, 'in_low.json'), 'w') as f:
    f.write(json.dumps(d_low, sort_keys=True, indent=2, separators=(',', ': ')))

In [ ]:
PlantLoop('in_low.json').simulate()

In [ ]:
df_low = pd.read_csv('out_low.csv', index_col=0, parse_dates=True)

In [ ]:
df_low['Outlet Temp Error [C]'] = df_low['GroundHeatExchangerSTS:GHE 1:Outlet Temp. [C]'] - df_raw_data_low['Exp. Outlet [C]']
df_low['Heat Rate Error [W]'] = df_low['GroundHeatExchangerSTS:GHE 1:Heat Rate [W]'] - df_raw_data_low['Exp. HT Rate [W]']
df_low['DT Error [C]'] = (df_low['GroundHeatExchangerSTS:GHE 1:Inlet Temp. [C]'] - df_low['GroundHeatExchangerSTS:GHE 1:Outlet Temp. [C]']) - (df_raw_data_low['Exp. Inlet [C]'] - df_raw_data_low['Exp. Outlet [C]'])
df_low['Heat Rate Error [%]'] = (df_low['GroundHeatExchangerSTS:GHE 1:Heat Rate [W]'] - df_raw_data_low['Exp. HT Rate [W]']) / df_raw_data_low['Exp. HT Rate [W]'] * 100

In [ ]:
def plot_temps_low(chart_no):

    fig = plt.figure(dpi=200)
    ax = fig.add_subplot(211)
    
    if chart_no == 1:
        start_time = '2019-01-01 00:00:00'
        end_time = '2019-01-01 00:30:00'
    elif chart_no == 2:
        start_time = '2019-01-01 00:00:00'
        end_time = '2019-01-01 02:00:00'
    elif chart_no == 3:
        start_time = '2019-01-01 01:00:00'
        end_time = '2019-01-02 18:00:00'
        
    ax.plot(df_raw_data_low['Exp. Inlet [C]'].loc[start_time:end_time], label='Exp. Inlet')
    ax.plot(df_low['GroundHeatExchangerSTS:GHE 1:Inlet Temp. [C]'].loc[start_time:end_time], linestyle='--', label='Sim. Inlet')
    
    ax.plot(df_raw_data_low['Exp. Outlet [C]'].loc[start_time:end_time], label='Exp. Outlet')
    ax.plot(df_low['GroundHeatExchangerSTS:GHE 1:Outlet Temp. [C]'].loc[start_time:end_time], linestyle='--', label='Sim. Outlet')

    plt.ylabel('Temperature [C]')
    plt.grid()
    plt.legend()
    ax.legend(loc=0, ncol=2)
    
    if chart_no in [1, 2]:
        ax.xaxis.set_major_formatter(mdates.DateFormatter("%H-%M"))
    elif chart_no in [3]:
        ax.xaxis.set_major_formatter(mdates.DateFormatter("%d"))
        
    ax2 = fig.add_subplot(212)
    ax2.plot(df_low['DT Error [C]'].loc[start_time:end_time], label=r'$\Delta$T Error')

    if chart_no in [1, 2]:
        ax2.xaxis.set_major_formatter(mdates.DateFormatter("%H-%M"))
        plt.xlabel('Hr-Min')
    elif chart_no in [3]:
        ax2.xaxis.set_major_formatter(mdates.DateFormatter("%d"))
        plt.xlabel('Day')

    plt.ylabel('Error [C]')

    plt.grid()
    
    if chart_no == 1:
        plt.savefig('Temps_Low_First-30min_ts-{}.PNG'.format(timestep), bbox_inches='tight')
    elif chart_no == 2:
        plt.savefig('Temps_Low_First-2hr_ts-{}.PNG'.format(timestep), bbox_inches='tight')
    elif chart_no == 3:
        plt.savefig('Temps_Low_Full_ts-{}.PNG'.format(timestep), bbox_inches='tight')
    plt.show()

In [ ]:
plot_temps_low(1)

In [ ]:
plot_temps_low(2)

In [ ]:
plot_temps_low(3)